<a href="https://colab.research.google.com/github/soycong/Recommendation-system/blob/main/recommendation_system_dg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
rating_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings.csv')
movie_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies.csv')

# 데이터 살펴보기

In [ ]:
rating_data.head()

In [ ]:
movie_data.head()

In [ ]:
movie_data.drop('genres', axis = 1, inplace = True)
movie_data.head()

In [ ]:
user_movie_data = pd.merge(rating_data, movie_data, on = 'movieId')
user_movie_data.head()
#user_movie_data.shape

# timestamp
# 'hour'만 남기기

In [ ]:
user_movie_data.timestamp

In [ ]:
from datetime import datetime

user_movie_data['timestamp']= user_movie_data['timestamp'].values.astype(dtype='int')

user_movie_data['timestamp']=[datetime.fromtimestamp(x).strftime('%H') for x in user_movie_data['timestamp']]


In [ ]:
#timestamp object 형태
user_movie_data

In [ ]:
user_movie_data['timestamp'].astype(int)

# 유저별 정렬
# 유저별 최빈값 시간 idsort

In [ ]:
idsort=user_movie_data.sort_values(by=['userId'])
idsort.head()

In [ ]:
#idsort=idsort['timestamp'].groupby(idsort['userId']).mean()

#df.groupby('gender')['region'].agg(**{'most_common_value':lambda x:x.mode()}).reset_index()

idsort=idsort.groupby('userId')['timestamp'].agg(lambda x:x.mode()).reset_index()


In [ ]:
idsort

In [ ]:
idsort.set_index('userId', inplace = True)

idsort

In [ ]:
print(len(idsort))

# 추천시스템

In [ ]:
user_movie_rating = user_movie_data.pivot_table('rating', index = 'userId', columns='title').fillna(0)

In [ ]:
user_movie_rating.head()

In [ ]:
movie_user_rating = user_movie_rating.values.T

In [ ]:
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(movie_user_rating)
matrix.shape

In [ ]:
matrix[0]

In [ ]:
corr = np.corrcoef(matrix)
corr.shape

In [ ]:
corr2 = corr[:200, :200]
corr2.shape

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(corr2)

In [ ]:
movie_title = user_movie_rating.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index("Guardians of the Galaxy (2014)")

In [ ]:
corr_coffey_hands  = corr[coffey_hands]
list(movie_title[(corr_coffey_hands >= 0.9)])[:50]

In [ ]:

df_ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings.csv')
df_movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies.csv')

In [ ]:
df_user_movie_ratings = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

In [ ]:
df_user_movie_ratings.head()

In [ ]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = df_user_movie_ratings.to_numpy()

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-영화에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [ ]:
matrix

In [ ]:
pd.DataFrame(matrix_user_mean, columns = df_user_movie_ratings.columns).head()

In [ ]:
# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [ ]:
sigma = np.diag(sigma)
sigma[0]

In [ ]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [ ]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_movie_ratings.columns)
df_svd_preds.head()

In [ ]:
df_svd_preds.shape

In [ ]:
def prtime(user_id):
    rtime=idsort.loc[[user_id],['timestamp']]
    return rtime

In [ ]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id - 1 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.userId == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_movies_df, on = 'movieId').sort_values(['rating'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_movies_df[~ori_movies_df['movieId'].isin(user_history['movieId'])]
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'movieId')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]

    #print(prtime(user_id))

    return user_history, recommendations

In [ ]:
already_rated, predictions = recommend_movies(df_svd_preds, 1, df_movies, df_ratings, 10)

In [ ]:
already_rated.head(10)

In [ ]:
predictions

In [ ]:
idsort.loc[[1],['timestamp']]

In [ ]:
idsort.loc[1]

# 유저마다 상위 영화 5개 data frame (user-prediction data frame) 

In [ ]:
print(predictions.head(5).title)

In [ ]:
titlelist=list(predictions.head(5).title)
print(titlelist)

In [ ]:
usern=len(idsort)+1
usern

In [ ]:
uppermovie=pd.DataFrame(columns=['title'])
uppermovie

In [ ]:
for i in range(1,usern):
    already_rated, predictions = recommend_movies(df_svd_preds, i, df_movies, df_ratings, 10)
    titlelist=list(predictions.head(5).title)
    uppermovie.loc[i]=[titlelist]

uppermovie


